In [1]:
import numpy as np
import math
from transforms3d.euler import euler2mat, mat2euler
from matplotlib import pyplot as plt
import os, cv2
import time
from PIL import Image

dataset = 21

K_cal = np.array([[585.05108211, 0, 242.94140713],
                    [0, 585.05108211, 315.83800193],
                      [0, 0, 1]])
K_cal_inv = np.linalg.inv(K_cal)

R_oc = np.zeros([4, 4])
R_oc[0:3, 0:3] = [[0, 0, 1],
                [-1, 0, 0],
                [0, -1, 0]]
R_oc[3, 3] = 1




def transformation(best_particle):
    '''
    create a the transformation matrix from body to world frame
    '''
    Rz = np.zeros([3,3])
    w_T_b = np.zeros((4, 4))
    theta = best_particle[2]
    Rz[0,0] = math.cos(theta)
    Rz[1,1] = math.cos(theta)
    Rz[2,2] = 1
    Rz[0,1] = -math.sin(theta)
    Rz[1,0] = math.sin(theta)
    w_T_b[0:3, 0:3] = Rz
    w_T_b[:, 3] = [best_particle[0], best_particle[1], 0, 1]
    return w_T_b


def sensor2body(pitch, yaw):
    R = euler2mat(0, pitch, yaw, axes='sxyz')
    b_T_c = np.zeros([4,4])
    b_T_c[0:3, 0:3] = R
    b_T_c[:, 3] = [0.18, 0.005, 0.36, 1]
    return b_T_c


def texturing(best_particle, RGB, MAP, depth_dt, tm):
    w_T_b = transformation(best_particle)
    w_T_b[2, 3] = 0.127  # z-axis height of center of gravity
    b_T_rgb = sensor2body(0.36, 0.021)
    w_T_c_rgb = np.dot(w_T_b, b_T_rgb)  # where w_T_b is from the particle filter
    deprgb = np.zeros([480, 640, 3])

    res_x = 6; res_y =8
    xsize = int(480/res_x); ysize=int(640/res_y)
    depth_dt = depth_dt[0:480:res_x, 0:640:res_y]
    i = np.arange(0, 480, res_x)
    j = np.arange(0, 640, res_y)
    dd = -0.00304*depth_dt + 3.31
    indvalid = np.where(dd > 0)
    depth = 1.03/dd
    
    rgbi = np.ceil((i.repeat(ysize).reshape(xsize, ysize) * 526.37 + dd*(-4.5*1750.46) + 19276.0)/585.051)
    rgbj = np.tile(np.ceil((j * 526.37 + 16662.0) / 585.051), xsize).reshape(xsize, ysize)

    deprgb[i[indvalid[0]], j[indvalid[1]], :] = RGB[np.int16(rgbi[indvalid]), np.int16(rgbj[indvalid]), :]
    i = i.repeat(ysize).reshape(xsize, ysize)
    j = j.repeat(xsize).reshape(xsize, ysize)
    u_v = np.stack((rgbi-240, rgbj-320, np.ones((xsize, ysize))), axis=2)
    temp = np.array(u_v).dot(K_cal_inv[None, None, :]).squeeze()
    sl = np.stack((depth, temp[:, :, 0] * depth, temp[:, :, 1] * depth, np.ones((xsize, ysize))), axis=2)
    temp2 = R_oc[None, None, :].dot(sl.reshape((xsize, ysize, 4, 1))).squeeze().swapaxes(2, 0).swapaxes(1, 0)
    sw = w_T_c_rgb[None, None, :].dot(temp2.reshape((xsize, ysize, 4, 1))).squeeze()
    ind_sw = np.where(np.logical_and(sw[2, :, :] < 1,sw[2, :, :]>-1))

    s_x = np.ceil((sw[0, ind_sw[0], ind_sw[1]] - MAP['xmin']) / MAP['res']).astype(np.int16) - 1
    s_y = np.ceil((sw[1, ind_sw[0], ind_sw[1]] - MAP['xmin']) / MAP['res']).astype(np.int16) - 1
    tm[s_x, s_y, :] = RGB[np.int16(rgbi[indvalid][ind_sw[0]]), np.int16(rgbj[indvalid][ind_sw[1]]), :]

    return tm




# texture

with np.load("Encoders%d.npz" % dataset) as data:
    encoder_counts = data["counts"]  # 4 x n encoder counts
    encoder_stamps = data["time_stamps"]  # encoder time stamps
with np.load("Kinect%d.npz" % dataset) as data:
    disp_stamps = data["disparity_time_stamps"]  # acquisition times of the disparity images
    rgb_stamps = data["rgb_time_stamps"]  # acquisition times of the rgb images
#with np.load('trajectory%d.npz' % dataset) as data:
 #   tract_xy = data['xy']
  #  tract_c = data['cell']
tract_xy = np.load("lyu/best_p_%d.npy" % dataset)
map_cm = np.load("lyu/map%d.npy"%dataset) 
stamps = encoder_stamps
MAP = {}
MAP['res'] = 0.1  # meters
MAP['xmin'] = -30  # meters
MAP['ymin'] = -30
MAP['ymax'] = 30
MAP['xmax'] = 30
MAP['sizex'] = int(np.ceil((MAP['xmax'] - MAP['xmin']) / MAP['res'] + 1))  # cells
MAP['sizey'] = int(np.ceil((MAP['ymax'] - MAP['ymin']) / MAP['res'] + 1))
tex_map = np.zeros((MAP['sizex'], MAP['sizey'], 3), dtype=np.uint8)

ts = time.time()

In [2]:
tract_xy = np.transpose(tract_xy)

In [3]:
print(tract_xy.shape)

(3, 4788)


In [ ]:
for i in range(stamps.shape[0] - 1):
    tb = time.time()
    if (stamps[i].squeeze() <= rgb_stamps[-1].squeeze()) and (stamps[i].squeeze() >= rgb_stamps[0].squeeze()):
        indx_r = np.argmin(abs(rgb_stamps - stamps[i]))
        indx_d = np.argmin(abs(disp_stamps - stamps[i]))
        img = Image.open('../dataRGBD/Disparity' + str(dataset) + '/disparity' + str(dataset) + '_' + str(indx_d + 1) + '.png')
        disparity_img = np.array(img.getdata(), np.uint16).reshape(img.size[1], img.size[0])
        RGB = cv2.imread('../dataRGBD/RGB' + str(dataset) + '/rgb' + str(dataset) + '_' + str(indx_d + 1) + '.png')
        RGB = cv2.cvtColor(RGB, cv2.COLOR_RGB2BGR)
        tex_map = texturing(tract_xy[:, i], RGB, MAP, disparity_img, tex_map)
        #plt.imshow(tex_map)
        #plt.pause(0.0001)
        #plt.title('Current iteration: %d' % i)
    te = time.time()
    print('Current iteration: %d, Total update took: %s sec, Expected time : %d min.\n' % (i, (te-ts), (te-tb)*stamps.shape[0]/60))

plt.figure()
plt.imshow(tex_map)

Current iteration: 0, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 1, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 2, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 3, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 4, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 5, Total update took: 0.0797431468963623 sec, Expected time : 0 min.

Current iteration: 6, Total update took: 0.08074045181274414 sec, Expected time : 0 min.

Current iteration: 7, Total update took: 0.08074045181274414 sec, Expected time : 0 min.

Current iteration: 8, Total update took: 0.08074045181274414 sec, Expected time : 0 min.

Current iteration: 9, Total update took: 0.08074045181274414 sec, Expected time : 0 min.

Current iteration: 10, Total update took: 0.08074045181274414 sec, Expected time : 0 min.

Current iterati

Current iteration: 140, Total update took: 0.32109951972961426 sec, Expected time : 18 min.

Current iteration: 141, Total update took: 0.4308030605316162 sec, Expected time : 8 min.

Current iteration: 142, Total update took: 0.5345263481140137 sec, Expected time : 8 min.

Current iteration: 143, Total update took: 0.6591932773590088 sec, Expected time : 9 min.

Current iteration: 144, Total update took: 0.7838594913482666 sec, Expected time : 9 min.

Current iteration: 145, Total update took: 0.9035382270812988 sec, Expected time : 9 min.

Current iteration: 146, Total update took: 1.0102548599243164 sec, Expected time : 8 min.

Current iteration: 147, Total update took: 1.1209585666656494 sec, Expected time : 8 min.

Current iteration: 148, Total update took: 1.2256779670715332 sec, Expected time : 8 min.

Current iteration: 149, Total update took: 1.3313958644866943 sec, Expected time : 8 min.

Current iteration: 150, Total update took: 1.4441332817077637 sec, Expected time : 8 min

Current iteration: 232, Total update took: 10.999513626098633 sec, Expected time : 8 min.

Current iteration: 233, Total update took: 11.11021637916565 sec, Expected time : 8 min.

Current iteration: 234, Total update took: 11.223912239074707 sec, Expected time : 9 min.

Current iteration: 235, Total update took: 11.35755729675293 sec, Expected time : 10 min.

Current iteration: 236, Total update took: 11.4822256565094 sec, Expected time : 9 min.

Current iteration: 237, Total update took: 11.600902557373047 sec, Expected time : 9 min.

Current iteration: 238, Total update took: 11.725568294525146 sec, Expected time : 9 min.

Current iteration: 239, Total update took: 11.833279609680176 sec, Expected time : 8 min.

Current iteration: 240, Total update took: 11.93401288986206 sec, Expected time : 7 min.

Current iteration: 241, Total update took: 12.035740375518799 sec, Expected time : 8 min.

Current iteration: 242, Total update took: 12.137468338012695 sec, Expected time : 8 min.

Cur

Current iteration: 324, Total update took: 21.07459831237793 sec, Expected time : 8 min.

Current iteration: 325, Total update took: 21.18826723098755 sec, Expected time : 9 min.

Current iteration: 326, Total update took: 21.298972368240356 sec, Expected time : 8 min.

Current iteration: 327, Total update took: 21.40870475769043 sec, Expected time : 8 min.

Current iteration: 328, Total update took: 21.516417503356934 sec, Expected time : 8 min.

Current iteration: 329, Total update took: 21.629088163375854 sec, Expected time : 8 min.

Current iteration: 330, Total update took: 21.743783473968506 sec, Expected time : 9 min.

Current iteration: 331, Total update took: 21.863462924957275 sec, Expected time : 9 min.

Current iteration: 332, Total update took: 21.96820878982544 sec, Expected time : 8 min.

Current iteration: 333, Total update took: 22.07589316368103 sec, Expected time : 8 min.

Current iteration: 334, Total update took: 22.170639753341675 sec, Expected time : 7 min.

Curr

Current iteration: 415, Total update took: 31.090808391571045 sec, Expected time : 8 min.

Current iteration: 416, Total update took: 31.19550132751465 sec, Expected time : 8 min.

Current iteration: 417, Total update took: 31.296231746673584 sec, Expected time : 8 min.

Current iteration: 418, Total update took: 31.41690969467163 sec, Expected time : 9 min.

Current iteration: 419, Total update took: 31.52063226699829 sec, Expected time : 8 min.

Current iteration: 420, Total update took: 31.634358167648315 sec, Expected time : 8 min.

Current iteration: 421, Total update took: 31.74702739715576 sec, Expected time : 8 min.

Current iteration: 422, Total update took: 31.856733322143555 sec, Expected time : 8 min.

Current iteration: 423, Total update took: 31.95849108695984 sec, Expected time : 8 min.

Current iteration: 424, Total update took: 32.069164991378784 sec, Expected time : 8 min.

Current iteration: 425, Total update took: 32.18186116218567 sec, Expected time : 8 min.

Curre

...

Current iteration: 1971, Total update took: 199.46019315719604 sec, Expected time : 8 min.

Current iteration: 1972, Total update took: 199.56890320777893 sec, Expected time : 8 min.

Current iteration: 1973, Total update took: 199.67462038993835 sec, Expected time : 8 min.

Current iteration: 1974, Total update took: 199.77734327316284 sec, Expected time : 8 min.

Current iteration: 1975, Total update took: 199.88405990600586 sec, Expected time : 8 min.

Current iteration: 1976, Total update took: 199.98581528663635 sec, Expected time : 8 min.

Current iteration: 1977, Total update took: 200.0954954624176 sec, Expected time : 8 min.

Current iteration: 1978, Total update took: 200.19323086738586 sec, Expected time : 7 min.

Current iteration: 1979, Total update took: 200.30094528198242 sec, Expected time : 8 min.

Current iteration: 1980, Total update took: 200.40865635871887 sec, Expected time : 8 min.

Current iteration: 1981, Total update took: 200.51736640930176 sec, Expected time

Current iteration: 2061, Total update took: 209.24006295204163 sec, Expected time : 8 min.

Current iteration: 2062, Total update took: 209.34475660324097 sec, Expected time : 8 min.

Current iteration: 2063, Total update took: 209.45047307014465 sec, Expected time : 8 min.

Current iteration: 2064, Total update took: 209.55818462371826 sec, Expected time : 8 min.

Current iteration: 2065, Total update took: 209.66091108322144 sec, Expected time : 8 min.

Current iteration: 2066, Total update took: 209.7805905342102 sec, Expected time : 9 min.

Current iteration: 2067, Total update took: 209.88331604003906 sec, Expected time : 8 min.

Current iteration: 2068, Total update took: 209.99501490592957 sec, Expected time : 8 min.

Current iteration: 2069, Total update took: 210.1017746925354 sec, Expected time : 8 min.

Current iteration: 2070, Total update took: 210.2134337425232 sec, Expected time : 8 min.

Current iteration: 2071, Total update took: 210.32313871383667 sec, Expected time :

Current iteration: 2153, Total update took: 219.2273244857788 sec, Expected time : 8 min.

Current iteration: 2154, Total update took: 219.3300461769104 sec, Expected time : 8 min.

Current iteration: 2155, Total update took: 219.43776082992554 sec, Expected time : 8 min.

Current iteration: 2156, Total update took: 219.55843782424927 sec, Expected time : 9 min.

Current iteration: 2157, Total update took: 219.69407415390015 sec, Expected time : 10 min.

Current iteration: 2158, Total update took: 219.82871222496033 sec, Expected time : 10 min.

Current iteration: 2159, Total update took: 219.9623568058014 sec, Expected time : 10 min.

Current iteration: 2160, Total update took: 220.0800437927246 sec, Expected time : 9 min.

Current iteration: 2161, Total update took: 220.18678426742554 sec, Expected time : 8 min.

Current iteration: 2162, Total update took: 220.28948163986206 sec, Expected time : 8 min.

Current iteration: 2163, Total update took: 220.39320540428162 sec, Expected time

Current iteration: 2243, Total update took: 229.0021777153015 sec, Expected time : 9 min.

Current iteration: 2244, Total update took: 229.12385320663452 sec, Expected time : 9 min.

Current iteration: 2245, Total update took: 229.22258949279785 sec, Expected time : 7 min.

Current iteration: 2246, Total update took: 229.338280916214 sec, Expected time : 9 min.

Current iteration: 2247, Total update took: 229.44499588012695 sec, Expected time : 8 min.

Current iteration: 2248, Total update took: 229.55170679092407 sec, Expected time : 8 min.

Current iteration: 2249, Total update took: 229.66739988327026 sec, Expected time : 9 min.

Current iteration: 2250, Total update took: 229.7791006565094 sec, Expected time : 8 min.

Current iteration: 2251, Total update took: 229.88381814956665 sec, Expected time : 8 min.

Current iteration: 2252, Total update took: 230.00250124931335 sec, Expected time : 9 min.

Current iteration: 2253, Total update took: 230.10725092887878 sec, Expected time : 

Current iteration: 2333, Total update took: 238.9316213130951 sec, Expected time : 8 min.

Current iteration: 2334, Total update took: 239.0383358001709 sec, Expected time : 8 min.

Current iteration: 2335, Total update took: 239.14903950691223 sec, Expected time : 8 min.

Current iteration: 2336, Total update took: 239.2587468624115 sec, Expected time : 8 min.

Current iteration: 2337, Total update took: 239.35748267173767 sec, Expected time : 7 min.

Current iteration: 2338, Total update took: 239.47416949272156 sec, Expected time : 9 min.

Current iteration: 2339, Total update took: 239.57792401313782 sec, Expected time : 8 min.

Current iteration: 2340, Total update took: 239.68463373184204 sec, Expected time : 8 min.

Current iteration: 2341, Total update took: 239.78932762145996 sec, Expected time : 8 min.

Current iteration: 2342, Total update took: 239.90701127052307 sec, Expected time : 9 min.

Current iteration: 2343, Total update took: 240.0127305984497 sec, Expected time : 

In [ ]:
plt.figure()
boolcm = (map_cm<0).reshape(map_cm.shape[0],map_cm.shape[1],1)

plt.imshow(tex_map*boolcm)
plt.savefig('lyu/texture21_1.png')